In [5]:
import win32gui
import win32process

def get_all_hwnds():
    hwnds_info = []
    def callback(hwnd, extra):
        if win32gui.IsWindowVisible(hwnd) and win32gui.GetWindowText(hwnd):
            title = win32gui.GetWindowText(hwnd)
            _, pid = win32process.GetWindowThreadProcessId(hwnd)
            hwnds_info.append((hwnd, title, pid))
        
        # Continue enumeration
        return True

    win32gui.EnumWindows(callback, None)
    return hwnds_info

if __name__ == "__main__":
    windows = get_all_hwnds()
    for hwnd, title, pid in windows:
        print(f"HWND: {hwnd}, PID: {pid}, Title: {title}")

HWND: 8259500, PID: 47852, Title: testing.ipynb - Pathway-IOTA - Visual Studio Code
HWND: 14420296, PID: 15084, Title: WhatsApp
HWND: 1311868, PID: 5460, Title: Google Gemini - Google Chrome
HWND: 4327808, PID: 5460, Title: v0 by Vercel - Google Chrome
HWND: 2296800, PID: 5460, Title: Restricted - Overleaf, Online LaTeX Editor - Google Chrome
HWND: 264316, PID: 5460, Title: Participation | Hacktoberfest 2025 - Google Chrome
HWND: 13765748, PID: 20656, Title: Realtek Audio Console
HWND: 12324008, PID: 1816, Title: Realtek Audio Console
HWND: 1184102, PID: 1816, Title: WhatsApp
HWND: 1573626, PID: 28032, Title: Settings
HWND: 2884192, PID: 1816, Title: Settings
HWND: 66378, PID: 16000, Title: Windows Input Experience
HWND: 65848, PID: 10632, Title: Program Manager


In [ ]:
def get_window_by_pid_and_title(pid: int, title_substring: str = "!!!"):
    windows = get_all_hwnds()
    for hwnd, title, win_pid in windows:
        if win_pid == pid or title_substring.lower() in title.lower():
            return hwnd, title, win_pid
    return None

def run_process(path: str):
    import subprocess
    import time

    proc = subprocess.Popen([path])
    print(f"Process started with PID: {proc.pid}")
    return proc

    # while True:
    #     retcode = proc.poll()
    #     if retcode is None:
    #         time.sleep(2)
    #     else:
    #         print(f"Process ended with exit code {retcode}")
    #         break

process_id = run_process("notepad.exe")
print(get_window_by_pid_and_title(process_id))

Process started with PID: 31600
None


In [28]:
import win32gui
import win32process
import psutil

def get_all_hwnds_by_ppid(ppid):
    hwnds_info = []
    def callback(hwnd, extra):
        if win32gui.IsWindowVisible(hwnd) and win32gui.GetWindowText(hwnd):
            title = win32gui.GetWindowText(hwnd)
            _, pid = win32process.GetWindowThreadProcessId(hwnd)

            try:
                proc = psutil.Process(pid)
                parent_pid = proc.ppid()
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                parent_pid = None

            hwnds_info.append((hwnd, title, pid, parent_pid))
        return True

    win32gui.EnumWindows(callback, None)
    return hwnds_info


run_process("notepad.exe")
if __name__ == "__main__":
    import os
    my_ppid = os.getpid()
    print("Looking for windows with parent PID:", my_ppid)


    windows = get_all_hwnds_by_ppid(my_ppid)
    for hwnd, title, pid, parent_pid in windows:
        print(f"HWND: {hwnd}, PID: {pid}, PPID: {parent_pid}, Title: {title}")


Process started with PID: 44104
Looking for windows with parent PID: 40672
HWND: 8259500, PID: 47852, PPID: 10632, Title: testing.ipynb - Pathway-IOTA - Visual Studio Code
HWND: 856724, PID: 19068, PPID: 10592, Title: Task Manager
HWND: 4327808, PID: 5460, PPID: 20788, Title: v0 by Vercel - Google Chrome
HWND: 1311868, PID: 5460, PPID: 20788, Title: wmic to get pid and ppid - Google Search - Google Chrome
HWND: 2296800, PID: 5460, PPID: 20788, Title: Restricted - Overleaf, Online LaTeX Editor - Google Chrome
HWND: 264316, PID: 5460, PPID: 20788, Title: Participation | Hacktoberfest 2025 - Google Chrome
HWND: 13765748, PID: 20656, PPID: 1532, Title: Realtek Audio Console
HWND: 12324008, PID: 1816, PPID: 1532, Title: Realtek Audio Console
HWND: 1573626, PID: 28032, PPID: 1532, Title: Settings
HWND: 2884192, PID: 1816, PPID: 1532, Title: Settings
HWND: 66378, PID: 16000, PPID: 1532, Title: Windows Input Experience
HWND: 65848, PID: 10632, PPID: 10852, Title: Program Manager


In [ ]:
import subprocess   
import time
import win32gui
import win32process

def find_main_window_for_pid(pid, timeout=10):
    """
    Finds the main, visible window handle (HWND) for a given process ID (PID).
    It polls for a specified timeout period because the window might not be
    created instantly after the process starts.
    """
    start_time = time.time()
    hwnd = None
    
    while (time.time() - start_time) < timeout:
        # Check for the window
        hwnd = _find_hwnd_for_pid(pid)
        if hwnd:
            return hwnd
        # Wait a moment before polling again
        time.sleep(0.25)
        
    return None # Return None if not found within timeout

def _find_hwnd_for_pid(target_pid):
    """Helper function to be called by find_main_window_for_pid."""
    result_hwnd = None

    def callback(hwnd, hwnds):
        nonlocal result_hwnd
        # Check if the window is visible and has a title
        if win32gui.IsWindowVisible(hwnd) and win32gui.GetWindowText(hwnd):
            # Get the PID of the process that owns this window
            _, found_pid = win32process.GetWindowThreadProcessId(hwnd)
            if found_pid == target_pid:
                result_hwnd = hwnd
                return False # Stop enumerating
        return True # Continue enumerating

    win32gui.EnumWindows(callback, [])
    return result_hwnd


def launch_and_rename_window(exe_path, new_title):
    """
    Launches an executable, finds its window, and renames it.
    Returns True on success, False on failure.
    """
    try:
        # 1. Launch the process
        print(f"🚀 Launching {exe_path}...")
        proc = subprocess.Popen([exe_path])
        launched_pid = proc.pid
        print(f"   -> Process started with PID: {launched_pid}")

        # 2. Find the window handle (HWND) for the new process
        print(f"   -> Searching for its main window...")
        hwnd = find_main_window_for_pid(launched_pid)

        if not hwnd:
            print(f"❌ Error: Could not find window for PID {launched_pid} within the timeout.")
            # We can terminate the process we started if we can't find its window
            proc.terminate()
            return False

        original_title = win32gui.GetWindowText(hwnd)
        print(f"   -> ✅ Found window! HWND: {hwnd}, Original Title: '{original_title}'")

        # 3. Set the new window title
        print(f"   -> Renaming window to '{new_title}'...")
        win32gui.SetWindowText(hwnd, new_title)
        print("   -> ✨ Success!")
        return True

    except FileNotFoundError:
        print(f"❌ Error: The application '{exe_path}' was not found.")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False


# --- Main Execution ---
if __name__ == "__main__":
    TARGET_APP = "notepad.exe"
    NEW_WINDOW_TITLE = "My Custom Notepad - Launched by Python"
    
    launch_and_rename_window(TARGET_APP, NEW_WINDOW_TITLE)

🚀 Launching notepad.exe...
   -> Process started with PID: 33160
   -> Searching for its main window...
❌ Error: Could not find window for PID 33160 within the timeout.
